<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#New-implementation-for-GPU" data-toc-modified-id="New-implementation-for-GPU-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>New implementation for GPU</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Saving-the-model" data-toc-modified-id="Saving-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>Saving the model</strong></a></span></li><li><span><a href="#Visualization" data-toc-modified-id="Visualization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Visualization</a></span></li></ul></div>

<a href="https://colab.research.google.com/github/arashfahim/LOB_multi-scale/blob/main/Impulse_3D_LOB_Forward_Non_Linear_Coarse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Running cost

$\sum_{t=0}^{T-1}D_t\xi_t+\frac{\kappa}{2}\xi^{1+\alpha}_t$

Terminal cost

$D_T(R_0-\sum_{t=0}^{T-1}\xi_t)+\frac{\kappa}{2}(R_0-\sum_{t=0}^{T-1}\xi_t)^{1+\alpha}$

For $T=2$, the total loss is

$D_0\xi_0+\frac{\kappa}{2}\xi^{1+\alpha}_0+D_1(R_0-\xi_0)+\frac{\kappa}{2}(R_0-\xi_0)^{1+\alpha}$

$D_1=\rho(D_0+\kappa \xi_0^{\alpha})$

loss$=D_0\xi_0+\frac{\kappa}{2}\xi^{1+\alpha}_0+\rho(D_0+\kappa \xi_0^{\alpha})(R_0-\xi_0)+\frac{\kappa}{2}(R_0-\xi_0)^{1+\alpha}$

$=D_0\xi_0+\frac{\kappa}{2}\xi^{1+\alpha}_0+\rho D_0R_0-\rho D_0\xi_0+\rho R_0\kappa \xi_0^{\alpha}-\rho \kappa \xi_0^{1+\alpha}+\frac{\kappa}{2}(R_0-\xi_0)^{1+\alpha}$



$=\rho D_0R_0+D_0(1-\rho)\xi_0 + \frac{\kappa}{2}(R_0^2-2(1-\rho)\xi_0(R_0-\xi_0))=\rho D_0R_0+\frac{\kappa}{2}R_0^2+D_0(1-\rho)\xi_0 - {\kappa}(1-\rho)\xi_0(R_0-\xi_0)$

$(1-\rho){\kappa}\xi_0(\frac{D_0}{\kappa} - R_0+\xi_0)$


$\xi_0(\frac{D_0}{\kappa} - R_0+\xi_0)$

------------------------------

Minimization:


$\xi_0^*=\frac{R_0-\frac{D_0}{\kappa}}{2}=\frac{\kappa R_0-{D_0}{}}{2\kappa}$

-------------------------------

If $D_0=0$, $\xi_0=\frac{R_0}{2}$.

Total cost


$D_0\frac{\kappa R_0-D_0}{2\kappa}+\frac{\kappa}{2}\frac{(\kappa R_0-D_0)^2}{4\kappa^2}+D_1(R_0-\frac{\kappa R_0-D_0}{2\kappa})+\frac{\kappa}{2}(R_0-\frac{\kappa R_0-D_0}{2\kappa})^2$


If $D_0=0$, $\frac{\kappa}{4}(1+\rho)R_0^2$


In [1]:
import numpy as np
import json
from scipy import misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import random
import math
import pandas as pd
# import cvxpy as cp
from scipy.optimize import fsolve
import time
import os

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
PATH = r"/Users/fahim/Downloads/"

# New implementation for GPU

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Parameters

In [5]:
param_dict = {
        'M' : 1000,# number of samples
        'T' : 1.0, # terminal time
        'iter' : 10, #number of steps minus terminal
        'kappa' : 1e-3, #price impact sensitivity factor
        'rho' : 5e0, #resillience
        'alpha' : 1.0, #price impact exponent
        'X0' : 10000, #focus initial balance
        'lr': 1e-2, #learning rate
}

In [6]:
class opt_exe(nn.Module):#multi_step, optimal_control
    def __init__(self,params):
        super(opt_exe, self).__init__()
        self.layer = torch.nn.Sequential()
        self.layer.add_module("L1",torch.nn.Linear(3, 20))
        self.layer.add_module("ReLU", torch.nn.ReLU())
        self.layer.add_module("L2",torch.nn.Linear(20,20))
        self.layer.add_module("Relu", torch.nn.ReLU())
        # self.layer.add_module("L3",torch.nn.Linear(256,128))
        # self.layer.add_module("Tanh", torch.nn.Tanh())
        self.layer.add_module("L4",torch.nn.Linear(20,1))
        self.M = params['M']# number of samples
        self.T= params['T'] # terminal time
        self.iter = params['iter'] #number of steps minus terminal
        self.Dt= torch.Tensor([self.T/self.iter]).to(device) #time step
        self.kappa=torch.Tensor([params['kappa']]).to(device) #price impact sensitivity factor
        self.rho=torch.exp(-params['rho']*self.Dt).to(device) #resillience<1
        self.alpha= params['alpha'] #price impact exponent
        self.X0=params['X0']
        self.D = torch.zeros([self.M,1]).to(device)
        self.R= torch.FloatTensor(self.M,1).uniform_(self.X0*0.9,self.X0*1.1).to(device)
        # self.R= torch.linspace(self.X0*0.9,self.X0*1.1,self.M).unsqueeze(-1).to(device)#torch.FloatTensor(self.M,1).uniform_(self.X0*0.95,self.X0*1.05).to(device)
        self.tx = torch.cat(((torch.Tensor([0.0]).repeat([self.M,1])).to(device),self.D,self.R),axis=1)
        self.opt_path = torch.zeros([self.M,self.iter+1,3]).to(device)
        self.opt_exe = torch.zeros([self.M,self.iter+1]).to(device)

    def forward(self, tx):
        val = self.layer(tx)
        return val#torch.Tensor([0.5]).to(device)*tx[:,-1]#val

    def unit(self,tx):
        exe_ = self.forward(tx)# \xi(t,X_t) optimal strategy
        co_ = (tx[:,1]*exe_.squeeze(-1)+torch.Tensor([1/(1+self.alpha)]).to(device)*self.kappa *torch.pow(torch.abs(exe_.squeeze(-1)),1+self.alpha)) #impulse control does not have Delta t
        up_ = torch.cat((tx[:,0].unsqueeze(-1)+self.Dt, self.rho*(tx[:,1].unsqueeze(-1)+self.kappa*torch.sign(exe_)*torch.pow(torch.abs(exe_),self.alpha)), tx[:,2].unsqueeze(-1)-exe_),axis=1) # continuous strategy needs Delta t in the amount executed
        return exe_, co_, up_

    def loss(self):
        for i in range(self.iter):
            if i == 0:
                #X_0=(0,0,R_0)
                tx = self.tx
                exe_, co_, up_ = self.unit(tx) # \xi(0,X_0), running cost for X_0, X_{t_1}
                cost = co_ #record running cost
            else:
                # self.tx[:,i,:] = up_
                exe_, co_, up_ = self.unit(up_) # \xi(t_i,X_{t_i}), running cost for X_{t_i}, X_{t_{i+1}}
                cost = cost + co_ # add the running cost
        cost = cost + (up_[:,1]*up_[:,-1]+torch.Tensor([1/(1+self.alpha)]).to(device)*self.kappa*torch.pow(torch.abs(up_[:,-1]),1+self.alpha)).squeeze(-1)   # cost of last order is impulse and do not need self.Dt*
        return torch.mean(cost)

    def loss2(self):
        for i in range(self.iter):
            if i == 0:
                #X_0=(0,0,R_0)
                tx = self.tx
                exe_, co_, up_ = self.unit(tx) # \xi(0,X_0), running cost for X_0, X_{t_1}
                # cost = co_ #record running cost
                xi = exe_
            else:
                # self.tx[:,i,:] = up_
                exe_, co_, up_ = self.unit(up_) # \xi(t_i,X_{t_i}), running cost for X_{t_i}, X_{t_{i+1}}
                xi = torch.concat((xi,exe_),axis=-1)
        xi = torch.concat((xi,up_[:,-1].unsqueeze(-1)),axis=-1)
        tau = self.rho.detach().cpu().numpy()
        Q = np.fromfunction(lambda i, j: np.power(tau,np.abs(i-j)), (self.iter+1, self.iter+1), dtype=float)
        Q = torch.Tensor(Q).to(device)
        cost = torch.Tensor([0.5]).to(device)*torch.bmm(torch.matmul(xi,Q).unsqueeze(1),xi.unsqueeze(-1))*self.kappa
        return torch.mean(cost)

    def approx_loss_2(self):
        for i in range(self.iter):
            if i == 0:
                self.opt_path[:,i,:] = self.tx.squeeze(-1)
                exe_, co_, up_ = self.unit(self.tx) # \xi(0,X_0), running cost for X_0, X_{t_1}
                cost = co_ #record running cost
                # sum = exe_ # record executed order
                # print(exe_)
                self.opt_exe[:,i] = exe_.squeeze(-1)
            else:
                self.opt_path[:,i,:] = up_
                exe_, co_, up_ = self.unit(up_) # \xi(t_i,X_{t_i}), running cost for X_{t_i}, X_{t_{i+1}}
                cost = cost + co_ # add the running cost
                self.opt_exe[:,i] = exe_.squeeze(-1)
        # last step
        self.opt_path[:,self.iter,:] = up_
        # print(exe_)
        remain = up_[:,-1] # remaining balance carried to the last order
        self.opt_exe[:,self.iter] = remain
        cost = cost + (up_[:,1]*remain+torch.Tensor([1/(1+self.alpha)]).to(device)*self.kappa *torch.pow(torch.abs(remain),1+self.alpha))   # cost of last order is impulse and do not need self.Dt*
        return torch.mean(cost)#torch.cat((tx[:,-1].unsqueeze(-1),cost.unsqueeze(-1)),axis=1)


    def optimal_path(self,tx):
        M = tx.shape[0]
        opt_path = torch.zeros([M,self.iter+1,3]).to(device)
        opt_exe = torch.zeros([M,self.iter+1]).to(device)
        for i in range(self.iter):
            if i == 0:
                opt_path[:,i,:] = tx.squeeze(-1)
                exe_, co_, up_ = self.unit(tx) # \xi(0,X_0), running cost for X_0, X_{t_1}
                cost = co_ #record running cost
                # sum = exe_ # record executed order
                # print(exe_)
                # print(i,":",exe_,",",up_[:,-1])
                opt_exe[:,i] = exe_.squeeze(-1)
            else:
                opt_path[:,i,:] = up_
                exe_, co_, up_ = self.unit(up_) # \xi(t_i,X_{t_i}), running cost for X_{t_i}, X_{t_{i+1}}
                cost = cost + co_ # add the running cost
                # sum = sum + exe_  # add the executed order to total
                # print(i,":",exe_,",",up_[:,-1])
                opt_exe[:,i] = exe_.squeeze(-1)
        # last step
        opt_path[:,self.iter,:] = up_
        # print(exe_)
        remain = up_[:,-1] # remaining balance carried to the last order
        # print("last: ",remain)
        opt_exe[:,self.iter] = remain
        cost = cost + (up_[:,1]*remain+torch.Tensor([1/(1+self.alpha)]).to(device)*self.kappa *torch.pow(torch.abs(remain),1+self.alpha))   # cost of last order is impulse and do not need self.Dt*
        return opt_path, opt_exe, cost#torch.cat((tx[:,-1].unsqueeze(-1),cost.unsqueeze(-1)),axis=1)

    def visualize(self,ind):
        self.approx_loss_2()
        self.eval_closed_form()
        h = []
        l = []
        color = cm.rainbow(np.linspace(0, 1, len(ind)))
        fig, ax = plt.subplots(len(ind),1,figsize=(len(ind)*2.5,5),dpi=300)
        for i in range(len(ind)):
            ax[i].bar(self.opt_path[ind[i],:,0].cpu().detach().numpy(),self.opt_exe[ind[i],:].clone().detach().cpu().numpy(),label=r"$R_0$="+str(self.R[ind[i],0].detach().cpu().numpy()),color=color[i],width=0.5*p1.Dt.cpu().numpy())
            h_, l_ = ax[i].get_legend_handles_labels()
            h = h + h_
            l = l + l_
        ax[0].legend(h,l,loc='best',bbox_to_anchor=(1.0, 0.1, 0.2, 0.5));#, );
        plt.tight_layout();

    def eval_closed_form(self):
        self.closed_form = dict()
        # output = dict()
        for index, X0 in enumerate(self.R):
            cls0 = [X0/(2+(self.iter-1)*(1-self.rho))]
            cls = [(1-self.rho)*cls0[0]]*(self.iter-1)
            cls_ = cls0
            cls = cls0+cls+cls_
            v = torch.Tensor(cls).to(device)
            # v = vt.detach().cpu().numpy()
            tau = self.rho.detach().cpu().numpy()
            Q = np.fromfunction(lambda i, j: np.power(tau,np.abs(i-j)), (self.iter+1, self.iter+1), dtype=float)
            Q = torch.Tensor(Q).to(device)
            cost = torch.Tensor([0.5]).to(device)*torch.matmul(torch.matmul(v,Q),v.reshape(self.iter+1,1))*self.kappa#impulse control does not have Delta t
            self.closed_form[index] = [X0,v,cost]


In [7]:
p1 = opt_exe(param_dict).to(device)

In [8]:
p1.eval_closed_form()

**path to save the outcome**

In [9]:
p1.tx[10,:].unsqueeze(0)

tensor([[   0.0000,    0.0000, 9012.4785]])

In [10]:
# p1(p1.tx)

In [11]:
p1.optimal_path(p1.tx)[0].shape,p1.optimal_path(p1.tx)[1].shape

(torch.Size([1000, 11, 3]), torch.Size([1000, 11]))

In [12]:
p1.approx_loss_2()

tensor(111885.8047, grad_fn=<MeanBackward0>)

In [13]:
num_epochs = 15000
optimizer1 = optim.Adam(p1.parameters(), param_dict['lr'])#,weight_decay=1e-2
L_ = torch.Tensor([-2000.0]).to(device)
cost = p1.loss()
loss_epoch = dict()
start=time.time()
duration = 0
epoch = 0
relerr = -1e-7

In [14]:
od = p1.state_dict()
arch = '3D'
for key, value in od.items():
    if 'bias' in key:
        arch = "-".join([arch,str(value.shape[0])])
nom = '_impulse'
for k,v in param_dict.items():
    nom = nom + '_' + k + '_' + str(v)
path = PATH + arch + nom
if not os.path.exists(path):
    os.mkdir(path)
with open(path+"/params.json", "w") as outfile:
    json_object = json.dumps(param_dict, indent=4)
    outfile.write(json_object)

In [15]:
while (torch.abs(L_-cost)/torch.abs(L_)>relerr) &  (epoch <= num_epochs):#
    optimizer1.zero_grad()
    cost=p1.loss()
    cost.backward()
    optimizer1.step()
    #   before_lr = optimizer1.param_groups[0]["lr"]
    #   scheduler1.step()
    #   after_lr = optimizer1.param_groups[0]["lr"]
    loss_epoch[epoch] = cost
    if epoch>0:
        L_ = loss_epoch[epoch-1]
    if (epoch % 1000==0):
        print("At epoch {} the cost is {}.".format(epoch,cost.detach()))
        # print("Epoch %d: SGD lr %.4f -> %.4f" % (epoch, before_lr, after_lr))
        # print("We are currently at cost {} versus analytic {}".format(p1.approx_loss_2(torch.Tensor([[0.0, 0.0, 10.0]]).to(device)),p1.analytic_loss_2(torch.Tensor([[10.0]]).to(device))))
    if  (torch.abs(L_-cost)/torch.abs(L_)<=relerr):
        print("Delta Loss = {} , epoch = {}".format(torch.abs(L_-cost)/torch.abs(L_),epoch))
        print("Final cost tally {}".format(p1.approx_loss_2(),cost))#,p1.analytic_loss_2(torch.Tensor([[10.0]]).to(device))
    epoch += 1
end= time.time()
duration += end-start
print("Learning time: {}".format(duration))

At epoch 0 the cost is 111885.8046875.
At epoch 1000 the cost is 14924.76171875.
At epoch 2000 the cost is 14915.61328125.
At epoch 3000 the cost is 14882.384765625.
At epoch 4000 the cost is 14783.400390625.
At epoch 5000 the cost is 14752.4140625.
At epoch 6000 the cost is 14735.0654296875.
At epoch 7000 the cost is 14703.2724609375.
At epoch 8000 the cost is 14798.09375.
At epoch 9000 the cost is 14572.65234375.
At epoch 10000 the cost is 14570.4892578125.
At epoch 11000 the cost is 14533.17578125.
At epoch 12000 the cost is 14510.9296875.
At epoch 13000 the cost is 14513.5859375.
At epoch 14000 the cost is 14512.4091796875.
At epoch 15000 the cost is 14511.931640625.
Learning time: 55.96561098098755


In [16]:
p1.loss(),p1.loss2()

(tensor(14510.8633, grad_fn=<MeanBackward0>),
 tensor(14510.8633, grad_fn=<MeanBackward0>))

In [17]:
p1.approx_loss_2()


tensor(14510.8633, grad_fn=<MeanBackward0>)

In [18]:
p1.loss(),p1.loss2()

(tensor(14510.8633, grad_fn=<MeanBackward0>),
 tensor(14510.8633, grad_fn=<MeanBackward0>))

In [19]:
loss_epoch_=dict()
for k,v in loss_epoch.items():
    loss_epoch_[k] = float(v.cpu().detach().numpy())

In [20]:
with open(path+"/progress_ckeckpoint.json", "w") as outfile:
    json_object = json.dumps(loss_epoch_, indent=4)
    outfile.write(json_object)

In [21]:
# pd.DataFrame(optimizer1.state_dict()).to_csv(path+"/optimal_path.csv", index=False)
# optimizer1.state_dict()

In [22]:
p1.approx_loss_2()

tensor(14510.8633, grad_fn=<MeanBackward0>)

In [23]:
p1.opt_path[50,:,-1]

tensor([9321.7988, 7653.6729, 6936.6460, 6254.6030, 5572.0215, 4888.8921,
        4205.2480, 3521.1199, 2852.3223, 2198.1089, 1552.7507],
       grad_fn=<SelectBackward0>)

In [24]:
p1.opt_exe[50,:].sum()

tensor(9321.7988, grad_fn=<SumBackward0>)

# **Saving the model**

In [25]:
torch.save(p1.state_dict(),path+"/model.pt")

In [26]:
# pd.DataFrame(p1.opt_path.cpu().detach().numpy()).to_csv(path+"/optimal_path.csv", index=False)
pd.DataFrame(p1.opt_exe.cpu().detach().numpy()).to_csv(path+"/optimal_exe.csv", index=False)

# Visualization

In [ ]:
p1.visualize([0,int(p1.M/4),int(p1.M/2),int(3*p1.M/4),p1.M-1])
plt.savefig(path+"/plots.svg")



In [ ]:
k = 500

In [ ]:
p1.closed_form[k]

In [ ]:
p1.optimal_path(p1.tx[k,:].unsqueeze(0))

In [ ]:
print("The percentage of relative error is {}%.".format((torch.abs(p1.optimal_path(p1.tx[k,:].unsqueeze(0))[-1]-p1.closed_form[k][-1])/p1.closed_form[k][-1]*100).detach().cpu().numpy()[0]))

In [ ]:
p1.tx[k,:]

In [ ]:
p1.loss()

In [ ]:
p1.loss2()